In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
%run damping_ratio.py

In [8]:
import glob
from time_of_file import time_of_file
files = glob.glob('dl_G345/datas/0.1hz/*.csv')
files.sort(key=time_of_file)

In [10]:
files

['dl_G345/datas/0.1hz\\dl_G345_1.4kn2017-8-16(10-46-19).csv',
 'dl_G345/datas/0.1hz\\dl_G345_2.8kn2017-8-16(10-48-36).csv',
 'dl_G345/datas/0.1hz\\dl_G345_3.5kn2017-8-16(10-50-42).csv',
 'dl_G345/datas/0.1hz\\dl_G345_4.2kn2017-8-16(10-52-50).csv',
 'dl_G345/datas/0.1hz\\dl_G345_4.9kn2017-8-16(10-54-59).csv',
 'dl_G345/datas/0.1hz\\dl_G345_5.6kn2017-8-16(10-57-9).csv',
 'dl_G345/datas/0.1hz\\dl_G345_6.3kn2017-8-16(10-59-8).csv',
 'dl_G345/datas/0.1hz\\dl_G345_7kn2017-8-16(11-1-9).csv',
 'dl_G345/datas/0.1hz\\dl_G345_0.7kn2017-8-16(11-3-45).csv',
 'dl_G345/datas/0.1hz\\dl_G345_2.1kn2017-8-16(11-5-46).csv',
 'dl_G345/datas/0.1hz\\dl_G345_7.7kn2017-8-16(11-8-11).csv',
 'dl_G345/datas/0.1hz\\dl_G345_8.4kn2017-8-16(11-10-24).csv']

In [6]:
df = pd.read_csv(f)
loop = np.stack((df['epsilon_d'].values[50:100],df['sigma_d'].values[50:100]),axis=-1)


In [7]:
for f in files:
    dp.data_process(f)